<h1 align=center><font size = 5>Capstone Project - Opportunistic Opening of a Gym</font></h1>

## Introduction

see separate Report
 

In [3]:
#Import necessary libraries
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



#### Create list with Restaurants
(Limit set on 30 restaurants, as there is a limit on the ratings that can be downloades for free per day)

In [33]:
#foursquare credentials
CLIENT_ID = 'UYLRJQTAUZZ3PAS3CNVFPLGRHBSVOFAP3CLVHOO0WPOLP2FS' #Foursquare ID
CLIENT_SECRET = 'WICKK4GYBDVLAWPKVFFSNB540GTMXIDE5YQHMJ55LCN4LHYA' #Foursquare Secret
VERSION = '20180604'
LIMIT = 30

In [34]:
#Define the center of Manhatten as Time Square

address = 'Time Square, New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

#manually adjusted latitude/longitude of Time Square
latitude = 40.758000
longitude = -73.985500


41.73835325 -74.03694730280651


In [35]:
#Search Restaurants in the close area (radius 1000 meters)
search_query = 'Restaurant'
radius = 1000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url
results = requests.get(url).json()
#results


In [36]:
#create dataframe (list of restaurants)

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
#dataframe.head()


In [37]:
#Clean Data
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

#Adding a column 'Restaurant' with the value 'True' if the category contains the word 'Restaurant'
dataframe_filtered['Restaurant'] = ''

for i, row in dataframe_filtered.iterrows():
      value = False
      if (dataframe_filtered.at[i,'categories']).find("Restaurant") !=-1:
        value = True
      dataframe_filtered.at[i,'Restaurant'] = value

dataframe_filtered = dataframe_filtered[dataframe_filtered['Restaurant'] == True]

#add column with entry-number
dataframe_filtered['entry']= range(1, len(dataframe_filtered) + 1)
dataframe_filtered['entry']=dataframe_filtered['entry'].astype(str)



# Result 1: List with all Restaurants close to the Center

In [38]:
#Export list of Restaurants to csv
dataframe_filtered.to_csv('01-All_Restaurants.csv', header=True)

# Get Rating and add it to Restaurant list
Does not work if quota of foursquare accesses was exceed (500 calls per day)

In [39]:
# Create list with Ratings for all Restaurants in Restaurant list
#ERROR MESSAGE WHEN CALLS IN FOURSQUARE EXCEEDS QUOTA

df_rating2 = pd.DataFrame(columns=['id', 'rating'])

for i, row in dataframe_filtered.iterrows():
    Rest_ID = dataframe_filtered.at[i,'id']
    Rest_ID = Rest_ID + '?'
    url = 'https://api.foursquare.com/v2/venues/{}&client_id={}&client_secret={}&v={}'.format(Rest_ID, CLIENT_ID, CLIENT_SECRET, VERSION)
    url
    results = requests.get(url).json()
    venue = results['response']['venue']
    df_loop = json_normalize(venue)
    df_rating2 = df_rating2.append(df_loop)


In [50]:
df_rating2[['rating']].mean()

rating    7.16875
dtype: float64

In [40]:
#combine restaurant list with rating list and export it as CSV

df_rating2[['id','rating']]
df_Rest_with_Rating = pd.merge(dataframe_filtered, df_rating2, how='left', on=['id', 'id'])
df_Rest_with_Rating=df_Rest_with_Rating[['entry','name_x','rating','id','lat','lng']]

In [41]:
#Exclude 'good' restaurants; good defined as rating > 7 (if rating not available = 'bad' restaurant)

df_Rest_with_Rating['Good'] = ''
ThresholdRating=7

for i, row in df_Rest_with_Rating.iterrows():
      value = False
      if df_Rest_with_Rating.at[i,'rating'] > ThresholdRating:
        value = True
      df_Rest_with_Rating.at[i,'Good'] = value

#drop 'good' restaurants
df_Rest_with_Rating_Final = df_Rest_with_Rating[df_Rest_with_Rating['Good'] == False]
df_Rest_with_Rating_Final

,entry,name_x,rating,id,lat,lng,Good
6,7,Ding BBQ and Hot Pot Restaurant,6.2,57041d93498eefdff07820df,40.760728,-73.986184,False
7,8,Chazz Palminteri Italian Restaurant,5.7,5638fc92cd10f9db71633582,40.756464,-73.980563,False
8,9,SOUTHGATE Bar & Restaurant,6.6,4bf3345798ac0f477aaa62a8,40.766262,-73.978536,False
9,10,Via Italia Restaurant,6.2,4aeccde8f964a5205bcb21e3,40.756888,-73.980923,False
10,11,Restaurant Triomphe,6.9,45692b08f964a520df3d1fe3,40.755800,-73.982215,False
11,12,525LEX Restaurant & Lounge,6.3,4e39cb96887777123dd78824,40.755744,-73.972663,False
12,13,Buenavista Restaurant & Bar,NaN,5bc7c23fa8eb60002c60c349,40.756867,-73.993513,False
15,16,The Garden Restaurant,6.9,5051fae4e889728152d8dfec,40.762188,-73.971606,False
16,17,Cancun Mexican Restaurant,6.1,4b79abadf964a520f5092fe3,40.765728,-73.983699,False


# Result 2: Export List with 'bad' Restaurants close to the Center
Does not work if quota of foursquare accesses was exceed (500 calls per day for ratings)

In [42]:
#export list with 'bad' Restaurant incl. information on Rating as CSV
df_Rest_with_Rating_Final.to_csv('02-Bad_Restaurants.csv', header=True)

In [43]:
#packages to show restaurants on map
!conda install -c conda-forge folium=0.5.0 --yes
import folium
from folium import plugins


Solving environment: ...working... done

# All requested packages already installed.



# Result 3: Map with all Restaurants in close area
(Does not exclude 'good' restaurants, if access quota for Foursquare access was reached)

In [44]:
# create map with all restaurants
rest_map = folium.Map(location=[latitude, longitude], zoom_start=15.5)

# instantiate a feature group for the restaurants in the dataframe
rest = folium.map.FeatureGroup()

# loop through the restaurants
for lat, lng, in zip(dataframe_filtered.lat, dataframe_filtered.lng):
    rest.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add pop-up text to each marker on the map

latitudes = list(dataframe_filtered.lat)
longitudes = list(dataframe_filtered.lng)
labels = list(dataframe_filtered.entry)

for lat, lng, label in zip(latitudes, longitudes, labels):
    folium.Marker([lat, lng], popup=label, icon=folium.Icon(color='blue', icon='spoon', prefix='fa')).add_to(rest_map)    
    
# add restaurants to map
rest_map.add_child(rest)

rest_map2 = rest_map.add_child(rest)
rest_map2.save('03-All_Restaurants_MAP.html')



# Result 4: Map with all 'bad' Restaurants in close area
(Does not exclude 'good' restaurants, if access quota for Foursquare access was reached)

In [45]:
# create map with all 'bad' restaurants
rest_map = folium.Map(location=[latitude, longitude], zoom_start=15.5)

# instantiate a feature group for the restaurants in the dataframe
rest = folium.map.FeatureGroup()

# loop through the restaurants
for lat, lng, in zip(df_Rest_with_Rating_Final.lat, df_Rest_with_Rating_Final.lng):
    rest.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='red',
            fill_opacity=0.6
        )
    )

latitudes = list(df_Rest_with_Rating_Final.lat)
longitudes = list(df_Rest_with_Rating_Final.lng)
labels = list(df_Rest_with_Rating_Final.entry)

for lat, lng, label in zip(latitudes, longitudes, labels):
    folium.Marker([lat, lng], popup=label, icon=folium.Icon(color='red', icon='spoon', prefix='fa')).add_to(rest_map)    
    
# add restaurants to map
rest_map.add_child(rest)

rest_map2 = rest_map.add_child(rest)
rest_map2.save('04-Bad_Restaurants_MAP.html')

# Result 5: Export list with 'bad' restaurants close to the center, not located next to existing Gym
(Does not exclude 'good' restaurants, if access quota for Foursquare access was reached)

In [46]:
search_query = 'Gym / Fitness Center'
radius = 500
#add field with distance to next gym

df_Rest_with_Rating_Final['distance_gym'] = ''

for i, row in df_Rest_with_Rating_Final.iterrows():
    latitude = df_Rest_with_Rating_Final.at[i,'lat']
    longitude = df_Rest_with_Rating_Final.at[i,'lng']
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
    url
    results3 = requests.get(url).json()
    gyms = results3['response']['venues']
    df_gyms = json_normalize(gyms)
    x = df_gyms['location.distance'].min()
    df_Rest_with_Rating_Final.at[i,'distance_gym']=x
    
df_Rest_with_Rating_Final_NoGym = df_Rest_with_Rating_Final['distance_gym']<100

df_Rest_with_Rating_Final_NoGym = df_Rest_with_Rating_Final[df_Rest_with_Rating_Final['distance_gym'] >100]

#export list with 'bad' Restaurant incl. information on Rating as CSV
df_Rest_with_Rating_Final_NoGym.to_csv('05-Bad_Restaurants_NoGym.csv', header=True)
   

C:\Users\matth\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


# Result 6: Map with 'bad' restaurants close to the center, not located next to existing Gym
(Does not exclude 'good' restaurants, if access quota for Foursquare access was reached)

In [47]:

rest_map = folium.Map(location=[latitude, longitude], zoom_start=15.5)

# instantiate a feature group for the restaurants in the dataframe
rest = folium.map.FeatureGroup()

# loop through the restaurants
for lat, lng, in zip(df_Rest_with_Rating_Final_NoGym.lat, df_Rest_with_Rating_Final_NoGym.lng):
    rest.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='green',
            fill_opacity=0.6
        )
    )

latitudes = list(df_Rest_with_Rating_Final_NoGym.lat)
longitudes = list(df_Rest_with_Rating_Final_NoGym.lng)
labels = list(df_Rest_with_Rating_Final_NoGym.entry)

for lat, lng, label in zip(latitudes, longitudes, labels):
    folium.Marker([lat, lng], popup=label, icon=folium.Icon(color='green', icon='spoon', prefix='fa')).add_to(rest_map)    
    
# add restaurants to map
rest_map.add_child(rest)

rest_map2 = rest_map.add_child(rest)
rest_map2.save('06-Bad_Restaurants_NoGym_MAP.html')